In [1]:
import numpy as np
import pandas as pd
import pickle

In [2]:
import pickle

f = open('/home/henry/Insight/Yogee/Datasets/Model_dataset/ModelDf.pckl', 'rb')
ModelDf = pickle.load(f)
f.close()

In [3]:
#Remove 2017 data
ModelDf = ModelDf[ModelDf['year'] != 2017]
ModelDf = ModelDf.reset_index(drop=True)

#Reduced land area size
ModelDf.loc[:,'LandArea'] = ModelDf.loc[:,'LandArea']/100000


In [4]:
#Get zipcodes in the dataset

#Load population dataset
Population2011FileLoc = "/home/henry/Insight/Yogee/Datasets/NY_Population_dataset/ACS_11_5YR_B01003/ACS_11_5YR_B01003_with_ann.csv"
Population2011Df = pd.read_csv(Population2011FileLoc)

#Get zipcodes to use
zipcodes = Population2011Df.loc[:,'GEO.id2']

In [5]:
#remove zipcodes with less than 100 people
for i in range(0,zipcodes.shape[0]):
    if Population2011Df.loc[i,'HD01_VD01'] < 100:
        zipcode = zipcodes[i]
        ModelDf = ModelDf[ModelDf['zip'] != zipcode]
        zipcodes = zipcodes.drop(i)
zipcodes = zipcodes.reset_index(drop=True)

In [6]:
#create test set
TestBin = np.random.random_sample(zipcodes.shape[0])>.9
TestData = pd.DataFrame([],columns=['population','FemaleRatio','Income','LandArea','PopDensity','TotalStudio','NewStudioNextYearBin'])
TestZipcodes = np.array([],dtype=int)
TrainValidData = pd.DataFrame([],columns=['population','FemaleRatio','Income','LandArea','PopDensity','TotalStudio','NewStudioNextYearBin'])
TrainValidZipcodes = np.array([],dtype=int)

for i in range(0,zipcodes.shape[0]):
        zipcode = zipcodes[i]
        zipcodeData = ModelDf[ModelDf['zip'] == zipcode]
        if TestBin[i]:
            TestData =  pd.concat([TestData, zipcodeData], axis=0,  sort=False)
            TestZipcodes = np.append(TestZipcodes,zipcode)
        else:
            TrainValidData =  pd.concat([TrainValidData, zipcodeData], axis=0 , sort=False)
            TrainValidZipcodes = np.append(TrainValidZipcodes,zipcode)

TestData = TestData.reset_index(drop=True)
TrainValidData = TrainValidData.reset_index(drop=True)

In [7]:
#create validation set
ValidBin = np.random.random_sample(TrainValidZipcodes.shape[0])>.7
ValidData = pd.DataFrame([],columns=['population','FemaleRatio','Income','LandArea','PopDensity','TotalStudio','NewStudioNextYearBin'])
ValidZipcodes = np.array([],dtype=int)
TrainData = pd.DataFrame([],columns=['population','FemaleRatio','Income','LandArea','PopDensity','TotalStudio','NewStudioNextYearBin'])
TrainZipcodes = np.array([],dtype=int)

for i in range(0,TrainValidZipcodes.shape[0]):
        zipcode = TrainValidZipcodes[i]
        zipcodeData = TrainValidData[TrainValidData['zip'] == zipcode]
        if ValidBin[i]:
            ValidData =  pd.concat([ValidData, zipcodeData], axis=0,  sort=False)
            ValidZipcodes = np.append(ValidZipcodes,zipcode)
        else:
            TrainData =  pd.concat([TrainData, zipcodeData], axis=0 , sort=False)
            TrainZipcodes = np.append(TrainZipcodes,zipcode)


ValidData = ValidData.reset_index(drop=True)
TrainData = TrainData.reset_index(drop=True)

In [8]:
#Separate predictor and target variables
TestDataX = TestData.loc[:,['population','FemaleRatio','Income','LandArea','PopDensity','TotalStudio']]
TestDataY = TestData.loc[:,['NewStudioNextYearBin']]
ValidDataX = ValidData.loc[:,['population','FemaleRatio','Income','LandArea','PopDensity','TotalStudio']]
ValidDataY = ValidData.loc[:,['NewStudioNextYearBin']]
TrainDataX = TrainData.loc[:,['population','FemaleRatio','Income','LandArea','PopDensity','TotalStudio']]
TrainDataY = TrainData.loc[:,['NewStudioNextYearBin']]

In [12]:
from sklearn import preprocessing
from sklearn import linear_model
from sklearn import svm
from sklearn import metrics
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import make_pipeline

RANDOM_STATE = 42

#Fill in missing data and scale input data
imputer = preprocessing.Imputer()
TestDataTransX = imputer.fit_transform(TestDataX)/10000
TestDataScaledX = preprocessing.scale(TestDataTransX).tolist()
ValidDataTransX = imputer.fit_transform(ValidDataX)/10000
ValidDataScaledX = preprocessing.scale(ValidDataTransX).tolist()
TrainDataTransX = imputer.fit_transform(TrainDataX)/10000
TrainDataScaledX = preprocessing.scale(TrainDataTransX).tolist()

TestDataValuesY = TestDataY.values.ravel().tolist()
ValidDataValuesY = ValidDataY.values.ravel().tolist()
TrainDataValuesY = TrainDataY.values.ravel().tolist()

# Create a pipeline for Linear Regression
pipeline = make_pipeline(SMOTE(random_state=RANDOM_STATE),
                         linear_model.LogisticRegression())
pipeline.fit(TrainDataScaledX,TrainDataValuesY)
LogValidY = pipeline.predict(ValidDataScaledX)    
LogLoss = metrics.log_loss(ValidDataValuesY,LogValidY)
LogAcc = metrics.accuracy_score(np.round(ValidDataValuesY),np.round(LogValidY))
ConfusionMatrix = metrics.confusion_matrix(ValidDataValuesY, LogValidY)
ConfusionMatrix

array([[2203,  556],
       [  13,   36]])

In [15]:
np.sum(LogValidY)

592

In [1]:
import pip #needed to use the pip functions
for i in pip.get_installed_distributions(local_only=True):
    print(i)

AttributeError: module 'pip' has no attribute 'get_installed_distributions'